Extract text or tables by slicing method
<br> This is done by reading word doc as xml file, using Beautiful Soup (ref:https://carstenknoch.com/2020/02/qualitative-data-analysis-with-microsoft-word-comments-python-updated/)

Process: 
1. Using TKinter for GUI, choose the required word documents 
2. Open chosen file/s using context manager, then write file name as a header
3. Loop through files. Unzip, parse through BeautifulSoup4, get text only
4. Set REgex search pattern for Start and End
5. Slice text based on RE search pattern Start/End, and save text as variable
6. Write text into CSV file

In [ ]:
# Import libraries
import zipfile
import csv
from bs4 import BeautifulSoup as Soup
import tkinter as tk
from tkinter import filedialog
import re

# Show file selection dialog box
root = tk.Tk()
root.withdraw()
paths = filedialog.askopenfilenames()
root.update()

with open("/".join(paths[0].split("/")[0:-1])+" output.csv", "w", newline="", encoding="utf-8-sig") as f:
    csvwriter = csv.writer(f)
    print(paths)
    for path in paths:
        # Write a header line with file name and table header
        csvwriter.writerow([path.split("/")[-1],""])

        unzip = zipfile.ZipFile(path)
        soup = Soup(unzip.read("word/document.xml"), "lxml")
                # doc = unzip.read("word/document.xml").decode()
        text = soup.get_text()
        # print(text)
        # print(soup)

        # Setup REgex seach patterns, as many as you require
        pattern1_start = re.compile(r"REgex Search Here")
        pattern1_end = re.compile(r"REgex Search Here")

        # Try Except block to catch index errors at the end
        try: 
            # Finding the index for beginning and end based on the REgex search above
            first_table = [x.end() for x in pattern1_start.finditer(text)] # beginning of Point A, based on the END of the search pattern, e.g., Point A[end]
            first_table_end = [x.start() for x in pattern1_end.finditer(text)] # end of Point B, based on the START of the search pattern, e.g., [start]Point B
            print(first_table,first_table_end)

            # Slice the text at beginning:end
            text1 = text[first_table[0]:first_table_end[0]]
            print(text1)
            csvwriter.writerow(["".join(text1), ""])

        except IndexError:
            print(f"Index error at document: {path}")

csvwriter.writerow(["".join("Insert Text Here"), ""]) # Usually write your column headers in "Insert Text Here"


General Table extractor

Should work on any table due to REgex, and looping through _all_ the tables per word document

v3 of a table extractor that is more generalized

Applicable for 1 x n tables where it presents as 

Q
___


A
_______

Q
____

A

In [9]:
"""
https://stackoverflow.com/questions/53621696/how-to-start-from-second-index-for-for-loop - for loop, index slicing

"""
# Import libraries
from docx import Document
import re
import tkinter as tk
from tkinter import filedialog
import csv

# Show file selection dialog box
root = tk.Tk()
root.withdraw()
paths = filedialog.askopenfilenames()
root.update()

# Search patterns
text_search = r"REgex Search Here"

with open("/".join(paths[0].split("/")[0:-1])+" NTWoutput.csv", "w", newline="", encoding="utf-8-sig") as f:
    csvwriter = csv.writer(f)
    
    # Loop through each chosen file in TKinter
    for index in paths:
        # Write file name as header
        csvwriter.writerow([index.split("/")[-1],""])
        doc = Document(index)

        # Loop every table in the document
        for table in doc.tables:
            table_data = [[cell.text for cell in row.cells] for row in table.rows]

            # Change the search pattern to "select" a different table
            # Use a Try Except block to catch index errors
            try: 
                # Comparing each table data to see if REgex search is found within the variable
                if re.search(text_search, str(table_data)): # If TRUE
                    # for each item within the table data, take the first item, this would be the Question part of the table
                    for index in table_data[::2]:
                        clean1 = list(map(str.strip,index))
                        csvwriter.writerow(["".join(clean1), ""])
                        # print(clean1)

                    # for each item within the table data, take the second item, this would be the Answer part of the table
                    for index in table_data[1::2]:
                        clean2 = list(map(str.strip,index))
                        # print(clean2)
                        csvwriter.writerow(["".join(clean2), ""])

            except IndexError:
                print(f"Index error at document: {path}")


n x n tables


In [ ]:
"""
https://stackoverflow.com/questions/53621696/how-to-start-from-second-index-for-for-loop - for loop, index slicing

"""
# Import Libraries
from docx import Document
import re
import tkinter as tk
from tkinter import filedialog
import csv

# Show file selection dialog box
root = tk.Tk()
root.withdraw()
paths = filedialog.askopenfilenames()
root.update()

# Search patterns
text_search =r"REgex Search Here"

with open("/".join(paths[0].split("/")[0:-1])+" NTWoutput.csv", "w", newline="", encoding="utf-8-sig") as f:
    csvwriter = csv.writer(f)
    
    # Loop through each chosen file in TKinter
    for index in paths:
        # Write file name as header
        csvwriter.writerow([index.split("/")[-1],""])
        doc = Document(index)

        
        # Loop through every table within the document
        for table in doc.tables:
            table_data = [[cell.text for cell in row.cells] for row in table.rows]   

            try:    
                # Change the search pattern to "select" a different table
                if re.search(text_search, str(table_data)): # If TRUE
                    print(index.split("/")[-1])
                    # print(table_data)
                    for item in table_data:
                        print(item)
                        csvwriter.writerows([item]) #writerows to allow for writing set items [1,2,3] as columns
            
            except IndexError:
                print(f"Index error at document: {path}")
                        
            